In [1]:
import sys
import os
from pyemd import emd
import numpy as np
from gensim import models
from gensim.models import word2vec
import numpy.linalg as la
from gensim.models.word2vec import Word2Vec
import scipy.spatial as spt
from scipy.spatial import distance
from pyemd import emd
from sklearn.metrics import euclidean_distances
import itertools
import collections
from numpy import dot, zeros, dtype, array, sqrt,double,array,sqrt, sum as np_sum
import time

In [2]:
word_vec = None
array = None
global words_1
global words_2
global words_1_set
global words_2_set
global File
removed_words = ['sdkls']

In [3]:
W = np.memmap("data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())

In [4]:
vocab_dict = {w: k for k, w in enumerate(vocab_list)}

In [5]:
def centroidOfFile(words):
    fileWords = set(words)
    numberWords = len(fileWords)
    VecMatrixWords =np.zeros((numberWords, 300))
    centroid =np.zeros((1, 300))
    for j,t in enumerate(fileWords):
        VecMatrixWords[j] = (W[[vocab_dict[t]]])
    centroid =np.zeros((1, 300))
    for j in range(300):
        for i in range(numberWords):
            centroid[0][j] =  centroid[0][j]+(VecMatrixWords[i][j])
        centroid[0][j]=  centroid[0][j]/300
    return(centroid)

In [6]:
def FileOpen(file1,file2):  
    global woduplicates
    woduplicates = None
    with open(file1) as f:
        global words_1
        global words_1_set
        words_1 = f.read().split()
        words_1 = [token for token in words_1 if token in vocab_dict.keys()]
        words_1_set = set(words_1)
    with open(file2) as f:
        global words_2
        global words_2_set
        words_2 = f.read().split()
        words_2 = [token for token in words_2 if token in vocab_dict.keys()]
        words_2_set = set(words_2)
    
    woduplicates = set(words_1 + words_2)
    return woduplicates


In [7]:
def Dictionaty2Files(content, woduplicates):
    doc_len = len(content)
    d = []
    for i, t in enumerate(woduplicates):
        d.append(content.count(t) / float(doc_len))
    return d 

In [8]:
def distance_matrixRWMD():
    len1 = len(words_1_set)
    len2 = len(words_2_set)
    T = zeros((len1, len2), dtype=float)
    distance = zeros((len1, len2), dtype=float)
    for i, t1 in enumerate(words_1_set):
        for j, t2 in enumerate(words_2_set):
            distance[i][j] = sqrt(np_sum((W[[vocab_dict[t1]]] - W[[vocab_dict[t2]]])**2))
        minimum = np.argmin(distance[i])
        T[i,minimum] = d1[i]*distance[i][minimum]
    sum1 = T.sum()
    T = zeros((len2, len1), dtype=float)
    distance = zeros((len2, len1), dtype=float)
    for i, t1 in enumerate(words_2_set):
        for j, t2 in enumerate(words_1_set):
            distance[i][j] = sqrt(np_sum((W[[vocab_dict[t1]]] - W[[vocab_dict[t2]]])**2))
        minimum = np.argmin(distance[i])
        T[i,minimum] = d1[j]*distance[i][minimum]
    
    sum2 = T.sum()
    return max(sum1,sum2)

In [9]:
def distance_matrix():
    vocab_len = len(woduplicates)
    distance_matrix = zeros((vocab_len, vocab_len), dtype=float)
    for i, t1 in enumerate(woduplicates):
        for j, t2 in enumerate(woduplicates):
            distance_matrix[i][j] = sqrt(np_sum((W[[vocab_dict[t1]]] - W[[vocab_dict[t2]]])**2))
    return distance_matrix

In [10]:
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    numberOfFiles = len(r)
    return r

In [11]:
#path = os.path.join(os.path.expanduser('~'), 'Downloads','LAB', '20news-bydate-train','test')
path = os.path.join(os.path.expanduser('.'),'data','dataWMD20NEWS','docs')
fnames = list_files(path)
numberOfFiles = len(os.listdir(path))


In [ ]:
start = time.time()
queryDocument = input("Please enter query document path: ")
k=30
d_wcd = dict()
WMD_DICT = dict()
RWMD_DICT = dict()
filename1 = queryDocument
fnames = list_files(path)
numberFiles = len(fnames)
REMAIN = (numberFiles-k)
for fileName2 in fnames:
    filename2 = fileName2
    global woduplicates
    woduplicates = FileOpen(filename1,filename2)
    centroid1 = centroidOfFile(words_1)
    centroid2 = centroidOfFile(words_2)
    d = sqrt(np_sum((centroid1 - centroid2)**2))
    d_wcd.update({(filename2, d)})
dict_wcd = (sorted(d_wcd.items(), key=lambda x:x[1]))
k_wcd = dict_wcd[0:k]

all_wcd = dict_wcd[0:numberFiles]
remain_wcd = list(set(all_wcd) - set(k_wcd))
remainFiles_wcd = remain_wcd[0:REMAIN]
k_wcd_array =  np.asarray(k_wcd)
kfiles_sorted_wcd = k_wcd_array[:,0]
remain_wcd_array = np.asarray(remainFiles_wcd)
remain_sorted_wcd = remain_wcd_array[:,0]


Please enter query document path: /home/vishwani/Downloads/LAB/FINALsubmissionSVEN/data/dataWMD20NEWS/querydoc


In [ ]:
#FOR K FILES CALCULATE WMD WITH filename1
for fileName2 in kfiles_sorted_wcd:
    filename2 = fileName2
    global woduplicates
    woduplicates = FileOpen(filename1,filename2)
    if((len(words_1)!=0)and len(words_2)!=0):
        d1 = np.array(Dictionaty2Files(words_1, woduplicates), dtype=double)
        d2 = np.array(Dictionaty2Files(words_2, woduplicates), dtype=double)
        distance = distance_matrix()
        distanceWMD = emd(d2,d1,distance)
        WMD_DICT.update({(filename2,distanceWMD )})
    else:
        print("skipped")
        print(filename2)
sorted_K_WMD = (sorted(WMD_DICT.items(), key=lambda x:x[1]))
sorted_K_WMD_array = np.asarray(sorted_K_WMD[0:k])
print(sorted_K_WMD_array)
   

In [ ]:
for fileName2 in remain_sorted_wcd:
    filename2 = fileName2
    #print(filename2)
    global woduplicates
    woduplicates = FileOpen(filename1,filename2)
    if((len(words_1)!=0)and len(words_2)!=0):
        d1 = np.array(Dictionaty2Files(words_1, words_1_set), dtype=double)
        d2 = np.array(Dictionaty2Files(words_2, words_2_set), dtype=double)
        distanceRWMD = distance_matrixRWMD()
        RWMD_DICT.update({(filename2,distanceRWMD)})
    else:
        print("skipped")
        print(filename2)

sorted_remain_RWMD = (sorted(RWMD_DICT.items(), key=lambda x:x[1]))
sorted_remain_RWMD_array = np.asarray(sorted_remain_RWMD[0:REMAIN])
for i,j in enumerate(sorted_remain_RWMD_array[:,1]):
    if(sorted_K_WMD_array[k-1,1]>sorted_remain_RWMD_array[i,1]):
        filename2 = (sorted_remain_RWMD_array[i,0])
        global woduplicates
        woduplicates = FileOpen(filename1,filename2)
        if((len(words_1)!=0)and len(words_2)!=0):
            d1 = np.array(Dictionaty2Files(words_1, woduplicates), dtype=double)
            d2 = np.array(Dictionaty2Files(words_2, woduplicates), dtype=double)
            distance = distance_matrix()
            distanceWMD = emd(d2,d1,distance)
            if(float(sorted_K_WMD_array[k-1,1])>distanceWMD):
                sorted_K_WMD_array[k-1,0] = filename2
                sorted_K_WMD_array[k-1,1] = distanceWMD
        else:
            print("skipped")
            print(filename2)
print(sorted_K_WMD_array)

In [ ]:
sorted_K_WMD_array = np.asarray(sorted_K_WMD[0:k])
print(sorted_K_WMD_array)
print((end-start)/60)